<a href="https://colab.research.google.com/github/savancs/product_recommender/blob/numa_branch/product_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install opencv-python

In [1]:
pip install pytesseract

ERROR: Operation cancelled by user


In [ ]:
import cv2
import torch

In [2]:
import numpy as np
import pandas as pd

In [5]:
sephora_data = 'https://raw.githubusercontent.com/jjone36/Cosmetic/master/data/cosmetic.csv'
sephora_df = pd.read_csv(sephora_data)

In [7]:
sephora_df.head(300)

,Label,URL,brand,name,price,rank,skin_type,ingredients
0,moisturizing-cream-oils-mists,https://www.sephora.com/product/creme-de-la-me...,LA MER,Crème de la Mer,$175.00,4.1,"['✔ Normal\n', '✔ Oily\n', '✔ Combination\n', ...","Algae (Seaweed) Extract, Mineral Oil, Petrolat..."
1,moisturizing-cream-oils-mists,https://www.sephora.com/product/facial-treatme...,SK-II,Facial Treatment Essence,$179.00\n($214.00 value),4.1,"['✔ Normal\n', '✔ Oily\n', '✔ Combination\n', ...","-Pitera™: A bio-ingredient rich in vitamins, o..."
2,moisturizing-cream-oils-mists,https://www.sephora.com/product/protini-tm-pol...,DRUNK ELEPHANT,Protini™ Polypeptide Cream,$68.00,4.4,"['✔ Normal\n', '✔ Oily\n', '✔ Combination\n', ...",-9 Signal Peptide Complex \n-Pygmy Waterlily S...
3,moisturizing-cream-oils-mists,https://www.sephora.com/product/the-moisturizi...,LA MER,The Moisturizing Soft Cream,$175.00,3.8,"['✔ Normal\n', '✔ Oily\n', '✔ Combination\n', ...","Algae (Seaweed) Extract, Cyclopentasiloxane, P..."
4,moisturizing-cream-oils-mists,https://www.sephora.com/product/your-skin-but-...,IT COSMETICS,Your Skin But Better™ CC+™ Cream with SPF 50+,$38.00,4.1,"['✔ Full\n', '✔ Sensitive\n', '✔ Normal\n', '✔...",-Collagen: Supports skin elasticity for a yout...
...,...,...,...,...,...,...,...,...
295,moisturizing-cream-oils-mists,https://www.sephora.com/product/the-moisturizi...,LA MER,The Moisturizing Matte Lotion,$270.00,3.9,"['✔ Normal\n', '✔ Oily\n']","Water, Algae (Seaweed) Extract, Propanediol, S..."
296,moisturizing-cream-oils-mists,https://www.sephora.com/product/jasmine-green-...,HERBIVORE,Jasmine Green Tea Balancing Toner,$39.00,4.2,"['✔ Oily\n', '✔ Combination\n', '✔ Sensitive\n']",-Green Tea: Reduces oiliness; packed with anti...
297,moisturizing-cream-oils-mists,https://www.sephora.com/product/super-restorat...,CLARINS,Super Restorative Night Age Spot Correcting Re...,$136.00,4.5,[],-Organic Montpellier Rock-Rose: Targets the so...
298,moisturizing-cream-oils-mists,https://www.sephora.com/product/goat-milk-mois...,KATE SOMERVILLE,Goat Milk Moisturizing Cream,$65.00,4.1,"['✔ Normal\n', '✔ Oily\n', '✔ Combination\n', ...",-Lactose from Goat’s Milk: Soothes and moistur...


In [10]:
sephora_df["Label"].unique()

array(['moisturizing-cream-oils-mists', 'cleanser', 'facial-treatments',
       'face-mask', 'eye-treatment-dark-circle-treatment',
       'sunscreen-sun-protection'], dtype=object)

In [11]:
sephora_df["brand"].unique()

array(['LA MER', 'SK-II', 'DRUNK ELEPHANT', 'IT COSMETICS', 'TATCHA',
       "KIEHL'S SINCE 1851", 'FRESH', 'BELIF', 'SUNDAY RILEY', 'FARMACY',
       'FIRST AID BEAUTY', 'CLINIQUE', 'BAREMINERALS', 'SHISEIDO',
       'CHARLOTTE TILBURY', 'ORIGINS', 'OLEHENRIKSEN', 'JOSIE MARAN',
       'FARSÁLI', 'LANEIGE', 'DR. JART+', 'GLOW RECIPE', 'HERBIVORE',
       'TARTE', 'CAUDALIE', 'YOUTH TO THE PEOPLE', 'BIOSSANCE',
       'ESTÉE LAUDER', 'SON & PARK', 'AMOREPACIFIC', 'BOBBI BROWN',
       'ERBORIAN', 'SMASHBOX', 'GLAMGLOW', 'PETER THOMAS ROTH',
       'JACK BLACK', 'PHILOSOPHY', 'PERRICONE MD', 'LANCÔME', 'ALGENIST',
       'MAKE UP FOR EVER', 'KORRES', 'MURAD', 'GUERLAIN', 'DIOR',
       'KORA ORGANICS', 'DR. DENNIS GROSS SKINCARE', 'REN CLEAN SKINCARE',
       'KOPARI', 'CLARINS', 'NURSE JAMIE', 'LANCER', "L'OCCITANE",
       'KATE SOMERVILLE', 'SUPERGOOP!', 'SATURDAY SKIN',
       'SEPHORA COLLECTION', 'MILK MAKEUP', 'DR. BRANDT SKINCARE',
       'J.ONE', 'KAT VON D', 'TATA HARPER', 'GO

In [12]:
sephora_df["name"].unique()

array(['Crème de la Mer', 'Facial Treatment Essence',
       'Protini™ Polypeptide Cream', ..., 'Self Tan Dry Oil SPF 50',
       'Pro Light Self Tan Bronzing Mist',
       'DERMAPROTECT Daily Defense Broad Spectrum SPF 50+'], dtype=object)

In [13]:
sephora_df["price"].unique()

array(['$175.00', '$179.00\n($214.00 value)', '$68.00', '$38.00',
       '$60.00', '$72.00', '$29.50', '$325.00', '$45.00', '$47.00',
       '$105.00', '$58.00', '$90.00\n($123.00 value)', '$30.00', '$39.00',
       '$40.00', '$230.00', '$85.00', '$78.00', '$92.00', '$100.00',
       '$34.00', '$28.00', '$240.00', '$48.00', '$54.00', '$25.00',
       '$35.00', '$99.00', '$63.00', '$55.00', '$80.00', '$32.00',
       '$65.00', '$18.00', '$10.00', '$245.00', '$37.00', '$57.00',
       '$120.00', '$44.00', '$52.00', '$42.00', '$110.00', '$49.00',
       '$82.00', '$195.00', '$59.00', '$145.00', '$95.00', '$290.00',
       '$36.00', '$190.00', '$135.00', '$122.00', '$112.00', '$88.00',
       '$29.00', '$229.00', '$84.00', '$127.00', '$205.00', '$75.00',
       '$225.00', '$115.00', '$56.00', '$24.00', '$50.00', '$90.00',
       '$270.00', '$76.00', '$280.00', '$130.00',
       '$69.00\n($106.00 value)', '$125.00', '$68.00\n($96.00 value)',
       '$94.00', '$140.00', '$4.00', '$108.00', '

In [14]:
sephora_df["rank"].unique()

array([4.1, 4.4, 3.8, 4.2, 5. , 4.3, 4.5, 4.6, 3.9, 4. , 3.7, 4.8, 4.7,
       nan, 3.3, 3.6, 3.5, 4.9, 3.4, 0. , 2.8, 2. , 3.1, 2.6, 3. , 3.2,
       1. , 2.9, 2.7, 1.8])

In [15]:
sephora_df["skin_type"].unique()

array(["['✔ Normal\\n', '✔ Oily\\n', '✔ Combination\\n', '✔ Dry\\n', '✔ Sensitive\\n']",
       "['✔ Normal\\n', '✔ Oily\\n', '✔ Combination\\n', '✔ Dry\\n']",
       "['✔ Full\\n', '✔ Sensitive\\n', '✔ Normal\\n', '✔ Dry\\n', '✔ Combination\\n', '✔ Oily\\n', '✔ Natural\\n']",
       "['✔ Normal\\n', '✔ Oily\\n', '✔ Combination\\n', '✔ Sensitive\\n']",
       '[]', "['✔ Normal\\n', '✔ Oily\\n', '✔ Combination\\n']",
       "['✔ Normal\\n', '✔ Dry\\n', '✔ Sensitive\\n']",
       "['✔ Normal\\n', '✔ Combination\\n', '✔ Dry\\n']",
       "['✔ Normal\\n', '✔ Dry\\n']", "['✔ Combination\\n', '✔ Dry\\n']",
       "['✔ Oily\\n', '✔ Dry\\n', '✔ Sensitive\\n']",
       "['✔ Full\\n', '✔ Normal\\n', '✔ Dry\\n', '✔ Combination\\n', '✔ Oily\\n', '✔ Matte\\n']",
       "['✔ Normal\\n', '✔ Combination\\n', '✔ Dry\\n', '✔ Sensitive\\n']",
       "['✔ Oily\\n', '✔ Combination\\n']",
       "['✔ Medium\\n', '✔ Normal\\n', '✔ Dry\\n', '✔ Combination\\n', '✔ Oily\\n', '✔ Matte\\n']",
       "['✔ Full\\n'

In [16]:
sephora_df["ingredients"].unique()
#split method for separation

array(['Algae (Seaweed) Extract, Mineral Oil, Petrolatum, Glycerin, Isohexadecane, Microcrystalline Wax, Lanolin Alcohol, Citrus Aurantifolia (Lime) Extract, Sesamum Indicum (Sesame) Seed Oil, Eucalyptus Globulus (Eucalyptus) Leaf Oil, Sesamum Indicum (Sesame) Seed Powder, Medicago Sativa (Alfalfa) Seed Powder, Helianthus Annuus (Sunflower) Seedcake, Prunus Amygdalus Dulcis (Sweet Almond) Seed Meal, Sodium Gluconate, Copper Gluconate, Calcium Gluconate, Magnesium Gluconate, Zinc Gluconate, Magnesium Sulfate, Paraffin, Tocopheryl Succinate, Niacin, Water, Beta-Carotene, Decyl Oleate, Aluminum Distearate, Octyldodecanol, Citric Acid, Cyanocobalamin, Magnesium Stearate, Panthenol, Limonene, Geraniol, Linalool, Hydroxycitronellal, Citronellol, Benzyl Salicylate, Citral, Sodium Benzoate, Alcohol Denat., Fragrance.\n\nPlease be aware that ingredient lists may change or vary from time to time. Please refer to the ingredient list on the product package you receive for the most up to date list 